# OpenAI Parameters

# Overview
When making requests to OpenAI models, several parameters can be used to control the behavior and output of the model. \
Understanding these parameters helps in fine-tuning the responses to meet specific requirements, whether for generating text, answering questions, or any other use case.

For more detailed examples, refer to the official documentation [Azure OpenAI Service](https://learn.microsoft.com/en-us/azure/ai-services/openai/reference)


In [2]:
pip install requests

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.4.1-cp312-cp312-win_amd64.whl.metadata (36 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached charset_normalizer-3.4.1-cp312-cp312-win_amd64.whl (102 kB)
Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install tiktoken

  Using cached tiktoken-0.8.0-cp312-cp312-win_amd64.whl.metadata (6.8 kB)
  Using cached regex-2024.11.6-cp312-cp312-win_amd64.whl.metadata (41 kB)
Using cached tiktoken-0.8.0-cp312-cp312-win_amd64.whl (883 kB)
Using cached regex-2024.11.6-cp312-cp312-win_amd64.whl (273 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()


client = AzureOpenAI(
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        api_version="2024-02-01"
    )

CHAT_COMPLETIONS_MODEL = "gpt-4o-mini"
SEED = 123

# Parameter: max_tokens
**Description**: The maximum number of tokens to generate in the completion. \
**Default**: 16 \
**Example**: max_tokens=50

The token count of your prompt plus max_tokens can't exceed the model's context length. \
Most models have a context length of 2048 tokens (except for the newest models, which support 4096). Please refer to documentation.

In [17]:
def call_openai_with_max_tokens(max_tokens):
    response = client.chat.completions.create(
          model="gpt-4o-mini",
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "The best pet is a"}],
                    max_tokens=max_tokens
    )
    return response.choices[0].message.content

# Generate with different presence_penalty values
penalties = [16, 32, 60, 100]
for penalty in penalties:
    print(f"Max Tokens: {penalty}\n")
    print(call_openai_with_max_tokens(penalty))
    print("\n" + "-"*80 + "\n")

Max Tokens: 16

The best pet varies from person to person, as it depends on individual preferences,

--------------------------------------------------------------------------------

Max Tokens: 32

The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Here are a few popular pets and what makes them great:

1. **

--------------------------------------------------------------------------------

Max Tokens: 60

The best pet varies depending on individual preferences, lifestyles, and living situations. Some people may find that dogs are the best pets because they are loyal, affectionate, and great companions. Others might prefer cats for their independence and playful nature. 

Small pets like rabbits, hamsters, or guinea pigs can

--------------------------------------------------------------------------------

Max Tokens: 100

The best pet is often subjective and varies from person to person based on individual preferences, lifestyle, and

# Parameter: temperature

**Description**: Controls the randomness of the output. Lower values make the output more deterministic, while higher values increase randomness. \
**Value Range**: 0 to 1 \
**Default Value**: 1 \
**Example**: temperature=0.7

Higher values means the model will take more risks. \
Try 0.9 for more creative applications, and 0 (argmax sampling) for ones with a well-defined answer.

---
**NOTE**: We generally recommend altering this or top_p but not both.


In [26]:
def call_openai(num_times, prompt, temperature=0.75, use_seed=False):
    for i in range(num_times):
        if use_seed:
            response = client.chat.completions.create(
                model= "gpt-4o-mini",
                messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "The best pet is a"}],
                    max_tokens=60,
                    seed=SEED,
                    temperature = 0.7
            )
        else:
            response = client.chat.completions.create(
                model= "gpt-4o-mini",
                messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "The best pet is a"}],
                    max_tokens=60,
                    temperature = 0.3
            )
        print(response.choices[0].message.content)

In [27]:
# Without seed and temperature, the response is different each time
call_openai(10, 'The best pet is a ')

The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Here are a few popular options:

1. **Dogs**: Known for their loyalty and companionship, dogs are often considered "man's best friend." They require regular exercise and social interaction.

2. **Cats**:
The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Some popular options include:

1. **Dogs**: Known for their loyalty and companionship, dogs are often considered "man's best friend." They require regular exercise and social interaction.

2. **Cats**: Cats are
The best pet is subjective and depends on individual preferences, lifestyles, and needs. Some people might argue that dogs are the best pets due to their loyalty and companionship. Others might prefer cats for their independence and low maintenance. Small animals like hamsters or rabbits can be great for those with limited space,
The best pet can vary greatly depending on indivi

In [28]:
# Now using a seed and 0 temperature, the response is the much more consisitent
call_openai(10, 'The best pet is a ', temperature = 0, use_seed=True)

The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Here are some popular options:

1. **Dogs**: Known for their loyalty and companionship, dogs are great for active individuals or families. They require regular exercise, training, and social interaction.

2. **Cats
The best pet can vary greatly depending on personal preferences, lifestyles, and living situations. Here are some popular options:

1. **Dogs**: Known for their loyalty and companionship, dogs are great for active individuals or families. They require regular exercise, training, and social interaction.

2. **Cats
The best pet can vary greatly depending on personal preferences, lifestyles, and living situations. Here are some popular options:

1. **Dogs**: Known for their loyalty and companionship, dogs are great for active individuals or families. They require regular exercise, training, and social interaction.

2. **Cats
The best pet can vary greatly depending on personal p

# Parameter: n
**Description**: Specifies the number of completions to generate for each prompt. \
**Default Value**: 1 \
**Example**: n = 3 

---
**Note**: Because this parameter generates many completions, it can quickly consume your token quota. Use carefully and ensure that you have reasonable settings for max_tokens and stop.

In [10]:
response = client.chat.completions.create(
                model= "gpt-4o-mini",
                messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "The best pet is a"}],
                    max_tokens=60,
                    seed=SEED,
                    temperature = 0.7
            )

for index, c in enumerate(response.choices):
    print(index, c.message.content)

0 The best pet can vary greatly depending on personal preferences, lifestyles, and living situations. Here are some popular options:

1. **Dogs**: Known for their loyalty and companionship, dogs are great for active individuals or families. They require regular exercise, training, and social interaction.

2. **Cats


# Parameter: presence_penalty
**Description**: Penalizes new tokens based on whether they appear in the text so far, encouraging the model to use new tokens. \
**Value Range**: -2.0 to 2.0 \
**Default Value**: 0 \
**Example**: presence_penalty=0.5

In [18]:
def call_openai_with_presence_penalty(presence_penalty):
    response = client.chat.completions.create(
          model=CHAT_COMPLETIONS_MODEL,
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "The best pet is a"}],
                    max_tokens=60,
                    presence_penalty= 0.5
    )
    return response.choices[0].message.content

# Generate with different presence_penalty values
penalties = [0, 0.5, 1.0, 1.5, 2.0]
for penalty in penalties:
    print(f"Presence Penalty: {penalty}\n")
    print(call_openai_with_presence_penalty(penalty))
    print("\n" + "-"*80 + "\n")

Presence Penalty: 0

The best pet is often subjective and depends on individual preferences, lifestyles, and living situations. Here are a few popular options and what makes them great:

1. **Dogs**: Known for their loyalty and companionship, dogs can be great for active individuals or families. They require regular exercise and social interaction

--------------------------------------------------------------------------------

Presence Penalty: 0.5

The best pet is subjective and can vary depending on individual preferences, lifestyle, and living situation. Here are a few popular options:

1. **Dogs** - Known for their loyalty and companionship, dogs can be great for active individuals or families.

2. **Cats** - More independent than dogs,

--------------------------------------------------------------------------------

Presence Penalty: 1.0

The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Here are some popular options:

1. **Do

# Parameter: frequency_penalty
**Description**: Penalizes new tokens based on their existing frequency in the text so far, reducing the likelihood of repeating the same line verbatim. \
**Value Range**: -2.0 to 2.0 \
**Default Value**: 0 \
**Example**: frequency_penalty=0.5

#### Use cases to explore
1. **Compare Responses** \
Generate multiple completions to compare and choose the best response for your use case.

2. **Increase Diversity** \
Use multiple completions to get a variety of responses, which is useful in creative applications.

3. **Enhance Robustness** \
Generate multiple responses to ensure consistency and accuracy across different completions.

#### Best Practices
1. **Optimize Prompt Length** \
Keep your prompts concise but informative to ensure the model has enough context.

2. **Adjust Temperature and Top_p** \
Use these parameters to balance between deterministic and creative responses.

3. **Monitor Token Usage** \
Be mindful of the max_tokens parameter to manage costs and response length.

4. **Use Stopping Sequences** \
Define stopping sequences to control where the model should stop generating text, ensuring the output is within the desired context.

5. **Generate Multiple Completions** \
Use the n parameter to generate multiple completions and select the best one for your needs.

## Exercise 1: Exploring 'top_p' Parameter
### Description:
- The 'top_p' parameter controls nucleus sampling. Lower values make the output more focused by limiting the pool of tokens.

### Task:
- Write a function that calls the model with different 'top_p' values and observe how the output changes.


In [19]:
def call_openai_with_top_p(top_p):
    response = client.chat.completions.create(
          model= "gpt-4o-mini",
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "Describe the benefits of exercise."}],
                    max_tokens=60,
                    top_p= 0.2
    )
    return response.choices[0].message.content

# Try with different values of top_p
top_p_values = [0.1, 0.3, 0.7, 1.0]
for value in top_p_values:
    print(f"Top P: {value}\n")
    print(call_openai_with_top_p(value))
    print("\n" + "-"*80 + "\n")


Top P: 0.1

Exercise offers a wide range of benefits for both physical and mental health. Here are some of the key advantages:

### Physical Health Benefits:
1. **Improved Cardiovascular Health**: Regular exercise strengthens the heart, improves circulation, and helps lower blood pressure and cholesterol levels.
2. **Weight Management

--------------------------------------------------------------------------------

Top P: 0.3

Exercise offers a wide range of benefits for both physical and mental health. Here are some of the key advantages:

### Physical Health Benefits:
1. **Improved Cardiovascular Health**: Regular exercise strengthens the heart, improves circulation, and helps lower blood pressure and cholesterol levels.
2. **Weight Management

--------------------------------------------------------------------------------

Top P: 0.7

Exercise offers a wide range of benefits for both physical and mental health. Here are some of the key advantages:

### Physical Health Benefits:
1.

## Exercise 2: Frequency Penalty Exploration
### Description:
- This exercise will explore how the 'frequency_penalty' parameter affects the model's tendency to repeat itself.

### Task:
- Generate completions using different frequency_penalty values and note the diversity in responses.



In [20]:
def call_openai_with_frequency_penalty(frequency_penalty):
    response = client.chat.completions.create(
          model=CHAT_COMPLETIONS_MODEL,
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "Tell me a fun fact about space."}],
                    max_tokens=60,
                    frequency_penalty=0.2
    )
    return response.choices[0].message.content

penalties = [-1.0, 0.0, 1.0, 2.0]
for penalty in penalties:
    print(f"Frequency Penalty: {penalty}\n")
    print(call_openai_with_frequency_penalty(penalty))
    print("\n" + "-"*80 + "\n")


Frequency Penalty: -1.0

A fun fact about space is that there are more stars in the universe than there are grains of sand on all the beaches on Earth! It's estimated that there are about 100 billion to 200 billion galaxies, each containing millions or even billions of stars, leading to an astonishing total that far exceeds the

--------------------------------------------------------------------------------

Frequency Penalty: 0.0

Sure! One fun fact about space is that it is completely silent. Unlike on Earth, sound waves need a medium, like air or water, to travel through. Since space is a near-perfect vacuum, there are very few particles to carry sound. So if you were floating in space, you wouldn't

--------------------------------------------------------------------------------

Frequency Penalty: 1.0

Sure! One fun fact about space is that a day on Venus lasts longer than a year on Venus. Venus has an extremely slow rotation on its axis, taking about 243 Earth days to complete o

## Exercise 3: Multi-Completion with 'n' Parameter
### Description:
- This exercise will help students generate multiple completions for the same prompt and compare the results.

### Task:
- Generate 3 completions for the same prompt by adjusting the 'n' parameter.


In [31]:
response = client.chat.completions.create(
    model="gpt-4o-mini",  
    messages=[{"role": "user", "content": "Quien es el mejor jugador de baloncesto de la historia"}],
    n=3
)

for index, c in enumerate(response.choices):
    print(index, c.message.content)

0 La pregunta de quién es el mejor jugador de baloncesto de la historia es muy subjetiva y a menudo genera debate entre aficionados y expertos. Algunos de los nombres más mencionados en esta conversación son:

1. **Michael Jordan**: Considerado por muchos como el mejor de todos los tiempos, Jordan ganó seis campeonatos de la NBA con los Chicago Bulls y fue nombrado MVP de las Finales en todas esas ocasiones. 

2. **LeBron James**: Con una carrera destacada que continúa, LeBron ha sido comparado con Jordan debido a su versatilidad, habilidades y logros en la NBA, incluido múltiples campeonatos y premios MVP.

3. **Kareem Abdul-Jabbar**: Es el máximo anotador en la historia de la NBA y ganó seis campeonatos, además de ser un hombre que revolucionó el juego con su famoso "skyhook".

4. **Bill Russell**: Con 11 campeonatos de la NBA, Russell es conocido por su defensa y liderazgo, marcando una era en el baloncesto.

5. **Magic Johnson y Larry Bird**: Ambos jugadores fueron icónicos en los 

## Exercise 4: Temperature vs. Deterministic Output
### Description:
- Adjust the 'temperature' parameter to explore the balance between creative and deterministic responses.

### Task:
- Generate completions with different temperature values and analyze the variance in output.


In [23]:
def call_openai_with_temperature(temperature):
    response = client.chat.completions.create(
          model=CHAT_COMPLETIONS_MODEL,
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "Suggest a name for a new tech startup."}],
                    max_tokens=30,
                    temperature= 0.6
    )
    return response.choices[0].message.content

temperatures = [0.0, 0.3, 0.7, 1.0]
for temp in temperatures:
    print(f"Temperature: {temp}\n")
    print(call_openai_with_temperature(temp))
    print("\n" + "-"*80 + "\n")

Temperature: 0.0

Sure! Here are some suggestions for a tech startup name:

1. **InnovateX**
2. **TechNest**
3. **Nex

--------------------------------------------------------------------------------

Temperature: 0.3

Here are several suggestions for a tech startup name:

1. **Innovatech**
2. **NexGen Solutions**
3. **TechFusion

--------------------------------------------------------------------------------

Temperature: 0.7

Sure! Here are several suggestions for a tech startup name:

1. **InnovateX**
2. **TechNest**
3. **QuantumLeap

--------------------------------------------------------------------------------

Temperature: 1.0

Sure! Here are a few suggestions for a tech startup name:

1. **TechNexus**
2. **InnoVate**
3.

--------------------------------------------------------------------------------

